In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import os
import scvelo as scv
import harmonypy as hm
from matplotlib.pyplot import rc_context
import re
import seaborn as sns
from adpbulk import ADPBulk
import diffxpy.api as de


2022-12-05 16:28:54.164761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 16:28:55.194745: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-05 16:28:56.972511: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/compiler/openmpi-3.6.1/lib64::/usr/local/cuda-10.1/lib64:/home/prakrithi/miniconda3/envs/scRNA_new/bin/:/home/prakrithi/miniconda3/envs/scRNA_new/lib/
2022-12-05 16:28:56.973814: W tensorflow/stream_executo

In [3]:
sc.settings.set_figure_params(dpi=150,fontsize=8)

# Load integrated and harmonized data

In [4]:
datapath = r"/nfs_master/prakrithi/abhay/testis_scripts/"
results_dir = '/nfs_master/prakrithi/abhay/testis_scripts/results_Nov25/'

In [5]:
adata = sc.read_h5ad(datapath + 'integrated_data.h5ad')

In [9]:
adata

AnnData object with n_obs × n_vars = 92108 × 48257
    obs: 'samples', 'percent_TE', 'percent_Alu', 'percent_AluY', 'percent_L1', 'percent_LINE', 'percent_LTR', 'percent_SVA', 'doublet', 'predicted', 'n_genes', 'percent_mito', 'percent_ribo', 'n_genes_by_counts', 'total_counts'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'log1p'

In [10]:
adata = adata.raw.to_adata()

In [11]:
adata.uns['log1p']["base"] = None

In [12]:
print(adata.X.shape)

(92108, 48257)


In [13]:
adata

AnnData object with n_obs × n_vars = 92108 × 48257
    obs: 'samples', 'percent_TE', 'percent_Alu', 'percent_AluY', 'percent_L1', 'percent_LINE', 'percent_LTR', 'percent_SVA', 'doublet', 'predicted', 'n_genes', 'percent_mito', 'percent_ribo', 'n_genes_by_counts', 'total_counts'
    var: 'n_cells', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'log1p'

In [19]:
df = adata.to_df()
df

,(CATTC)n,(GAATG)n,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,...,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,Zaphod,Zaphod2,Zaphod3,hAT-16_Crp,hAT-N1_Mam
AAACCTGAGAAACCTA-1-Donor1_r1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,2.0
AAACCTGAGAAAGTGG-1-Donor1_r1,0.0,0.0,0.0,0.0,0.0,8.0,5.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGACTGGGT-1-Donor1_r1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGATCCTGT-1-Donor1_r1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGATGGGTC-1-Donor1_r1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCCTCGTGTTAAGA-1-OA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTCCTCTCAAGATCC-1-OA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGGTTAGTGTACTC-1-OA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGGTTGTCATCGGC-1-OA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Add metadata for TE elements


In [14]:
#load list of human TEs
all_te = pd.read_csv('/nfs_master/prakrithi/abhay/testis_scripts/extra_files/all_TE.csv')
all_te = all_te['All_TEs'].tolist()
gene_list = list(adata.var.index)
te_list = list(set(all_te).intersection(gene_list))

In [15]:
#Alu = list(filter(lambda x:'Alu' in x, all_te))
#Alu

In [16]:
Alu  = list(filter(lambda x:'Alu' in x, te_list))
AluY = list(filter(lambda x:'AluY' in x, te_list))
L1   = list(filter(lambda x:'L1' in x, te_list))
LINE = list(filter(lambda x:'LINE' in x, te_list))
LTR  = list(filter(lambda x:'LTR' in x, te_list))
SVA  = list(filter(lambda x:'SVA' in x, te_list))

In [17]:
len(te_list)


667

# Dimensionality Reduction

#### Normalize and scale the data

In [14]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

#### Calculate highly variable genes

In [18]:
adata.var

,n_cells,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
(CATTC)n,3040,3040,0.110448,97.296817,12421.0
(GAATG)n,2108,2108,0.034510,98.125556,3881.0
A1BG,2077,2077,0.020914,98.153121,2352.0
A1BG-AS1,485,485,0.004499,99.568736,506.0
A1CF,119,119,0.001183,99.894185,133.0
...,...,...,...,...,...
Zaphod,20278,20278,0.326801,81.968700,36752.0
Zaphod2,7791,7791,0.089863,93.072203,10106.0
Zaphod3,12277,12277,0.151067,89.083230,16989.0
hAT-16_Crp,32,32,0.000285,99.971545,32.0


In [16]:
sc.pp.highly_variable_genes(adata,n_top_genes=3000)

In [17]:
adata_hvg = adata[:, adata.var['highly_variable']].copy()

In [18]:
sc.tl.pca(adata_hvg, svd_solver='arpack', n_comps=40)

In [19]:
sc.pl.pca_variance_ratio(adata_hvg, log=False)
plt.savefig(results_dir + 'elbow-plot.png', dpi = 300)

In [ ]:
adata_hvg.obs
adata.obs

#adata.obs["leiden"] = adata_hvg.obs["leiden"]

#### Harmony Batch correction

In [20]:
pca = adata_hvg.obsm['X_pca']
batch = adata_hvg.obs['samples']
meta_data = adata_hvg.obs

In [21]:
ho = hm.run_harmony(pca, meta_data, ['samples'], epsilon_harmony = -float('Inf'), max_iter_kmeans = 25)


2022-11-25 15:48:11,951 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2022-11-25 15:49:39,561 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2022-11-25 15:51:03,044 - harmonypy - INFO - Iteration 3 of 10
INFO:harmonypy:Iteration 3 of 10
2022-11-25 15:52:25,662 - harmonypy - INFO - Iteration 4 of 10
INFO:harmonypy:Iteration 4 of 10
2022-11-25 15:53:43,918 - harmonypy - INFO - Iteration 5 of 10
INFO:harmonypy:Iteration 5 of 10
2022-11-25 15:54:24,322 - harmonypy - INFO - Iteration 6 of 10
INFO:harmonypy:Iteration 6 of 10
2022-11-25 15:55:43,548 - harmonypy - INFO - Iteration 7 of 10
INFO:harmonypy:Iteration 7 of 10
2022-11-25 15:56:44,348 - harmonypy - INFO - Iteration 8 of 10
INFO:harmonypy:Iteration 8 of 10
2022-11-25 15:57:17,279 - harmonypy - INFO - Iteration 9 of 10
INFO:harmonypy:Iteration 9 of 10
2022-11-25 15:58:38,687 - harmonypy - INFO - Iteration 10 of 10
INFO:harmonypy:Iteration 10 of 10
2022-11-25 15:59:08,294 - harmonypy - 

In [22]:
res = pd.DataFrame(ho.Z_corr)
res = res.T
adata_hvg.obsm['X_pca'] = res.values


# Clustering

In [ ]:
#sc.pp.normalize_total(adata_hvg, target_sum=1e4)
#sc.pp.log1p(adata_hvg)
#adata.raw=adata

In [23]:
sc.pp.neighbors(adata_hvg,n_neighbors=30)

In [24]:
sc.tl.umap(adata_hvg,spread=0.8,min_dist=0.3)

In [25]:
sc.tl.leiden(adata_hvg,resolution=0.45, n_iterations=-1)

In [26]:
sc.pl.umap(adata_hvg,color=["leiden","samples"],cmap="plasma_r")
plt.savefig(results_dir + 'unannotated-umap.png', dpi = 300)

# Differential Gene Expression Analysis

In [27]:
sc.tl.rank_genes_groups(adata_hvg,pts=True,groupby='leiden',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [28]:
sc.tl.dendrogram(adata_hvg,groupby="leiden")

In [29]:
with rc_context({'figure.figsize': (8, 8)}):
    sc.pl.rank_genes_groups_dotplot(adata_hvg,n_genes=7,
                                    groupby="leiden",color_map="plasma_r")
plt.savefig(results_dir + 'integrated-dotplot-unsupervised.png')


In [30]:
markers = sc.get.rank_genes_groups_df(adata_hvg, None)
markers = markers[(markers.pvals_adj < 0.05) & (markers.logfoldchanges > 0.5)]
markers.to_csv(results_dir + 'DEG_by_cluster.csv')


In [31]:
deg_df = pd.read_csv('DEG_by_cluster.csv')
deg_df = deg_df.iloc[: , 1:]
deg_df = deg_df[(abs(deg_df.logfoldchanges) > 0.25) & (deg_df.pvals_adj < 0.05)]
deg_df

,group,names,scores,logfoldchanges,pvals,pvals_adj,pct_nz_group,pct_nz_reference
0,0,AC007557.3,196.216920,5.184291,0.000000,0.000000,0.966552,0.338873
1,0,LINC01921,195.963040,5.166825,0.000000,0.000000,0.966359,0.341982
2,0,TNP1,191.507640,4.481681,0.000000,0.000000,0.986225,0.683292
3,0,CRISP2,122.572044,2.652664,0.000000,0.000000,0.878970,0.496745
4,0,NUPR2,120.452760,2.663200,0.000000,0.000000,0.855914,0.451098
...,...,...,...,...,...,...,...,...
1146,12,GADD45G,4.673165,0.946252,0.000003,0.008898,0.223856,0.110825
1147,12,NDRG4,4.576270,2.387134,0.000005,0.014200,0.135621,0.028679
1148,12,KLHL4,4.522102,4.026890,0.000006,0.018369,0.112745,0.006831
1149,12,MBNL3,4.511675,2.079002,0.000006,0.019295,0.143791,0.038078


# Cluster Annotation  

In [32]:
sc.pl.umap(adata_hvg, color=["leiden", "samples"], frameon = False)
plt.savefig(results_dir +'unannotated-umap.png')


In [33]:
#markers of interest
macrophage         = ['CD14', 'CD163', 'S100A4']     
endothelial        = ['PALMD', 'VWF', 'CDH5']        
other              = ['RBP1', 'INSL3', 'MYL9']       
lydig              = ['DLK1','IGF1','CFD']           
myoid              = ['MYH11','ACTA2','TPM4']        
sertoli            = ['SOX9', 'WT1', 'HMGN5']        
#immature_spermatid = ['ZPBP', 'ZPBP2', 'SPAG6']
#spermatid          = ['SPATA18', 'HOOK1', 'SPATA12'] 

#spermatogonia
spermatogonia      = ['UTF1', 'ID4', 'SOHLH1']  
undiff_SPG         = ['MAGEA4','ID4','TCF3']
diff_SPG           = ['SOX4','DMRT1']

#spermatocytes
leptotene          = ['TEX19','DPH7','DMC1']
zygotene           = ['MLH3','SELENOT', 'TDRG1','LY6K']
pachytene          = ['PIWIL1','CCDC112']
diplotene          = ['AURKA', 'CCNA1']
meotic_div         = ['SIRPG', 'SLC26A3']

round_spt          = ['SPAG6']
elong_spt          = ['ZPBP', 'ZPBP2']
#elong_spt          = ['ZPBP', 'ZPBP2', 'DNAH6', 'DNAH7']
early_spt          = ['TEX29']
late_spt           = ['PRM3','SPATA12']

na = ['AURKA', 'CCDC112', 'CCNA1', 'CDH5', 'DMC1', 'DMRT1', 'DPH7', 'ID4', 'MAGEA4', 'MLH3', 
 'PALMD', 'PIWIL1', 'PRM3', 'SELENOT', 'SPAG6', 'SPATA12', 'TCF3', 'TEX19', 'ZPBP', 'ZPBP2']

In [34]:
#marker lists
spermatogonia = undiff_SPG + diff_SPG
spermatids = round_spt + elong_spt + early_spt + late_spt
pre_pachytene = leptotene + zygotene 
post_pachytene = pachytene + diplotene 
somatic_m = macrophage + lydig + sertoli + endothelial
 
best = spermatids + post_pachytene + spermatogonia + pre_pachytene + sertoli + lydig + endothelial + macrophage



In [35]:
#test marker dotplot for annotation
sc.pl.dotplot(adata, best, groupby='leiden', dendrogram=True)
plt.savefig(results_dir +'test-dotplot.png')

ValueError: groupby has to be a valid observation. Given leiden, is not in observations: ['samples', 'percent_TE', 'percent_Alu', 'percent_AluY', 'percent_L1', 'percent_LINE', 'percent_LTR', 'percent_SVA', 'doublet', 'predicted', 'n_genes', 'percent_mito', 'percent_ribo', 'n_genes_by_counts', 'total_counts']

In [36]:
label_dict = {'0': 'Elongating_spermatids',
              '1': 'Lydig_cells',
              '2': 'Round_spermatids',
              '3': "Post_pachytene_spermatocytes",
              '4': "Mature_spermatids",
              '5': "Unknown_1",
              '6': "Spermatogonia",
              '7': "Pre_pachytene_spermatocytes",
              '8': "Sertoli_cells",
              '9': "Endothelial_cells",
             '10': "Macrophages",
             '11': "Unknown_2",
             '12': "Unknown_3"
             } 
adata_hvg.obs['clusters'] = adata_hvg.obs['leiden'].map(label_dict).astype('category')

In [37]:
#labelled dotplot
sc.pl.dotplot(adata_hvg, best, groupby='clusters', dendrogram=True)
plt.savefig('labelled-dotplot.png')

KeyError: "Could not find keys '['AURKA', 'CCDC112', 'CCNA1', 'CDH5', 'DMC1', 'DMRT1', 'DPH7', 'ID4', 'MAGEA4', 'MLH3', 'PALMD', 'PIWIL1', 'PRM3', 'SELENOT', 'SPAG6', 'SPATA12', 'TCF3', 'TEX19', 'ZPBP', 'ZPBP2']' in columns of `adata.obs` or in adata.var_names."

In [40]:
#spermatocyte-alu heatmap
sc.pl.heatmap(adata_hvg, Alu, groupby='clusters', swap_axes=False, dendrogram=True)
plt.savefig('integrated-alu-heatmap.png')

KeyError: "Could not find keys '['AluJb', 'AluJo', 'AluJr', 'AluJr4', 'AluSc', 'AluSc5', 'AluSc8', 'AluSg', 'AluSg4', 'AluSg7', 'AluSp', 'AluSq', 'AluSq10', 'AluSq2', 'AluSq4', 'AluSx', 'AluSx1', 'AluSx3', 'AluSx4', 'AluSz', 'AluSz6', 'AluY', 'AluYa5', 'AluYb8', 'AluYb9', 'AluYd8', 'AluYe5', 'AluYf1', 'AluYg6', 'AluYh9', 'AluYi6', 'AluYk11', 'AluYk12']' in columns of `adata.obs` or in adata.var_names."

In [41]:
#labelled dotplot
sc.pl.dotplot(adata_hvg, Alu, groupby='clusters', dendrogram=True)
plt.savefig('integrated-alu-dotplot.png')

KeyError: "Could not find keys '['AluJb', 'AluJo', 'AluJr', 'AluJr4', 'AluSc', 'AluSc5', 'AluSc8', 'AluSg', 'AluSg4', 'AluSg7', 'AluSp', 'AluSq', 'AluSq10', 'AluSq2', 'AluSq4', 'AluSx', 'AluSx1', 'AluSx3', 'AluSx4', 'AluSz', 'AluSz6', 'AluY', 'AluYa5', 'AluYb8', 'AluYb9', 'AluYd8', 'AluYe5', 'AluYf1', 'AluYg6', 'AluYh9', 'AluYi6', 'AluYk11', 'AluYk12']' in columns of `adata.obs` or in adata.var_names."

In [42]:
#labelled UMAP
sc.pl.umap(adata_hvg, color=["clusters"], frameon = False, legend_loc = 'on data', legend_fontsize = 'xx-small')
plt.savefig(results_dir +'labelled-umap.png')

# Subset germcells

In [ ]:
germcells = adata_hvg[adata_hvg.obs['leiden'].isin(['0', '2', '3', '4', '6', '7'])].copy()
germcells


In [ ]:
germcells.write_h5ad('germcells.h5ad')

In [ ]:
germcells.obs

# Subset spermatocytes

In [ ]:
spermatocytes = adata_hvg[adata_hvg.obs['leiden'].isin(['3', '7'])]

In [ ]:
spermatocytes.obs

# Analysis 

In [47]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [48]:
def map_condition(x):
    if "Donor" in x:
        return "Normal"
    elif "Normal" in x:
        return "Normal"
    elif "Cry" in x:
        return "Cryptozoospermia"
    elif "iNOA" in x:
        return "Non-obstructive_azoospermia"
    else:
        return "obstructive_azoospermia"

In [49]:
adata_hvg.obs['condition'] = adata_hvg.obs.samples.map(map_condition)
adata_hvg.obs

,samples,percent_TE,percent_Alu,percent_AluY,percent_L1,percent_LINE,percent_LTR,percent_SVA,doublet,predicted,n_genes,percent_mito,percent_ribo,n_genes_by_counts,total_counts,leiden,clusters,condition
AAACCTGAGAAACCTA-1-Donor1_r1,Donor1_r1,0.310769,0.215581,0.034718,0.043784,0.000100,0.007920,0.001893,0.070411,False,5493,0.054094,0.00005,5493,20076.0,6,Spermatogonia,Normal
AAACCTGAGAAAGTGG-1-Donor1_r1,Donor1_r1,0.158619,0.110893,0.012914,0.020307,0.000094,0.001591,0.001778,0.050003,False,2923,0.032753,0.00000,2923,10686.0,9,Endothelial_cells,Normal
AAACCTGAGACTGGGT-1-Donor1_r1,Donor1_r1,0.131544,0.100671,0.005369,0.005369,0.000000,0.001342,0.000000,0.047673,False,411,0.193289,0.00000,411,745.0,0,Elongating_spermatids,Normal
AAACCTGAGATCCTGT-1-Donor1_r1,Donor1_r1,0.111072,0.065378,0.003163,0.006327,0.000000,0.003515,0.000000,0.064560,False,1488,0.025659,0.00000,1488,2845.0,0,Elongating_spermatids,Normal
AAACCTGAGATGGGTC-1-Donor1_r1,Donor1_r1,0.061363,0.045637,0.002158,0.004317,0.000000,0.001850,0.000617,0.035771,False,1684,0.133518,0.00000,1684,3243.0,7,Pre_pachytene_spermatocytes,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCCTCGTGTTAAGA-1-OA,OA,0.079515,0.024259,0.004043,0.002695,0.000000,0.004043,0.000000,0.034958,False,375,0.000000,0.00000,375,742.0,0,Elongating_spermatids,obstructive_azoospermia
TTTCCTCTCAAGATCC-1-OA,OA,0.038835,0.007767,0.000000,0.003883,0.000000,0.000000,0.000000,0.241935,False,304,0.128155,0.00000,304,515.0,0,Elongating_spermatids,obstructive_azoospermia
TTTGGTTAGTGTACTC-1-OA,OA,0.032050,0.006290,0.000899,0.002247,0.000000,0.003744,0.000150,0.096902,False,1827,0.012131,0.00000,1827,6677.0,0,Elongating_spermatids,obstructive_azoospermia
TTTGGTTGTCATCGGC-1-OA,OA,0.359563,0.264684,0.066265,0.033133,0.000000,0.012048,0.006401,0.024532,False,1166,0.045181,0.00000,1166,2656.0,7,Pre_pachytene_spermatocytes,obstructive_azoospermia


In [50]:
num_tot_cells = adata_hvg.obs.groupby(['samples']).count()
num_tot_cells = dict(zip(num_tot_cells.index, num_tot_cells.percent_TE))

num_tot_cells

{'Donor1_r1': 4392,
 'Donor1_r2': 5860,
 'Donor2_r1': 4552,
 'Donor2_r2': 5292,
 'Donor3_r1': 5694,
 'Donor3_r2': 7369,
 'Normal_1': 9914,
 'Normal_2': 9525,
 'Normal_3': 9501,
 'Crypto_1': 6563,
 'Crypto_2': 6507,
 'Crypto_3': 9774,
 'iNOA_1': 1274,
 'iNOA_2': 622,
 'iNOA_3': 4140,
 'OA': 1129}

In [51]:
cell_type_counts = adata_hvg.obs.groupby(['samples','condition', 'clusters']).count()
cell_type_counts = cell_type_counts[cell_type_counts.sum(axis =1) > 0].reset_index()
cell_type_counts = cell_type_counts[cell_type_counts.columns[0:4]]
cell_type_counts = cell_type_counts.rename(columns = {'percent_TE':'cell_count'})

cell_type_counts.to_csv('germcell_count.csv')

cell_type_counts

,samples,condition,clusters,cell_count
0,Donor1_r1,Normal,Elongating_spermatids,701
1,Donor1_r1,Normal,Lydig_cells,703
2,Donor1_r1,Normal,Round_spermatids,411
3,Donor1_r1,Normal,Post_pachytene_spermatocytes,590
4,Donor1_r1,Normal,Mature_spermatids,339
...,...,...,...,...
190,OA,obstructive_azoospermia,Pre_pachytene_spermatocytes,113
191,OA,obstructive_azoospermia,Sertoli_cells,60
192,OA,obstructive_azoospermia,Endothelial_cells,7
193,OA,obstructive_azoospermia,Macrophages,5


In [52]:
cell_type_counts['total_cells'] = cell_type_counts.samples.map(num_tot_cells).astype(int)

cell_type_counts['frequency'] = cell_type_counts.cell_count/ cell_type_counts.total_cells

cell_type_counts

,samples,condition,clusters,cell_count,total_cells,frequency
0,Donor1_r1,Normal,Elongating_spermatids,701,4392,0.159608
1,Donor1_r1,Normal,Lydig_cells,703,4392,0.160064
2,Donor1_r1,Normal,Round_spermatids,411,4392,0.093579
3,Donor1_r1,Normal,Post_pachytene_spermatocytes,590,4392,0.134335
4,Donor1_r1,Normal,Mature_spermatids,339,4392,0.077186
...,...,...,...,...,...,...
190,OA,obstructive_azoospermia,Pre_pachytene_spermatocytes,113,1129,0.100089
191,OA,obstructive_azoospermia,Sertoli_cells,60,1129,0.053144
192,OA,obstructive_azoospermia,Endothelial_cells,7,1129,0.006200
193,OA,obstructive_azoospermia,Macrophages,5,1129,0.004429


In [53]:
import matplotlib.pyplot as plt

plt.figure(figsize = (15,7))

ax = sns.boxplot(data = cell_type_counts, x = 'clusters', y = 'frequency', hue = 'condition')

plt.xticks(rotation = 35, rotation_mode = 'anchor', ha = 'right')

plt.show()
plt.savefig(results_dir + 'integrated_freq_plot')

# DGE of germcells 

In [ ]:
germcells.obs['condition'] = germcells.obs.samples.map(map_condition)
germcells.obs

In [ ]:
sc.tl.rank_genes_groups(germcells,pts=True,groupby='condition',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [ ]:
sc.tl.dendrogram(germcells,groupby="condition")

In [ ]:
TE_of_interest = intersection(te_list, adata_hvg.var.index)
TE_of_interest

In [ ]:
#Alu expression by condition in gremcells
sc.pl.dotplot(germcells, TE_of_interest, groupby='condition', dendrogram=True)
plt.savefig('germcell-alu-dotplot.png')

In [ ]:
sc.pl.heatmap(germcells, Alu, groupby='condition', swap_axes=False)
plt.savefig('germcells-alu-heatmap.png')

### DGE of spermatocytes by condition

In [ ]:
sc.tl.rank_genes_groups(spermatocytes,pts=True,groupby='condition',n_genes=100,method='wilcoxon',corr_method='bonferroni')

In [ ]:
sc.tl.dendrogram(spermatocytes,groupby="condition")

In [ ]:
spermatocytes.obs['condition'] = spermatocytes.obs.samples.map(map_condition)
spermatocytes.obs

In [ ]:
#Alu expression by condition in spermatocytes
sc.pl.dotplot(germcells, Alu, groupby='condition', dendrogram=True)
plt.savefig('spermatocytes-alu-dotplot.png')

In [ ]:
sc.pl.heatmap(germcells, Alu, groupby='condition', swap_axes=False)
plt.savefig('spermatocyte-alu-heatmap.png')

In [ ]:
adata_hvg.obs

# Pseudobulking

In [43]:
#initialize
adpb_sum  = ADPBulk(adata_hvg, ["clusters", "samples"], "sum")
adpb_mean = ADPBulk(adata_hvg, ["clusters", "samples"], "mean")

# perform the pseudobulking
pseudobulk_matrix_sum  = adpb_sum.fit_transform()
pseudobulk_matrix_mean = adpb_mean.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta_sum  = adpb_sum.get_meta()
sample_meta_mean = adpb_mean.get_meta()



Aggregating Samples: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 208/208 [00:00<00:00, 2312.17it/s]


In [44]:
pseudobulk_matrix_sum.T

,clusters.Elongating_spermatids-samples.Crypto_1,clusters.Elongating_spermatids-samples.Crypto_2,clusters.Elongating_spermatids-samples.Crypto_3,clusters.Elongating_spermatids-samples.Donor1_r1,clusters.Elongating_spermatids-samples.Donor1_r2,clusters.Elongating_spermatids-samples.Donor2_r1,clusters.Elongating_spermatids-samples.Donor2_r2,clusters.Elongating_spermatids-samples.Donor3_r1,clusters.Elongating_spermatids-samples.Donor3_r2,clusters.Elongating_spermatids-samples.Normal_1,...,clusters.Unknown_3-samples.Donor2_r2,clusters.Unknown_3-samples.Donor3_r1,clusters.Unknown_3-samples.Donor3_r2,clusters.Unknown_3-samples.Normal_1,clusters.Unknown_3-samples.Normal_2,clusters.Unknown_3-samples.Normal_3,clusters.Unknown_3-samples.OA,clusters.Unknown_3-samples.iNOA_1,clusters.Unknown_3-samples.iNOA_2,clusters.Unknown_3-samples.iNOA_3
(CATTC)n,44.222950,2.335455,106.130661,5.661070,38.044884,3.501247,4.535139,11.254840,13.926229,219.565948,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
(GAATG)n,11.835286,3.952145,31.792885,7.273436,27.482759,2.646257,5.498230,4.963284,6.594667,82.832062,...,0.000000,0.000000,0.0,0.000000,0.0,0.648364,0.0,0.000000,0.0,0.000000
A2M,21.947166,3.808043,111.423073,26.296461,49.228653,64.127113,69.253548,62.783024,82.897598,57.893936,...,1.153113,0.790152,0.0,0.000000,0.0,0.000000,0.0,0.241722,0.0,39.680496
AARD,58.717648,9.039034,17.058331,14.728537,21.259560,111.891563,131.250351,35.150112,71.232765,379.810486,...,0.000000,0.000000,0.0,4.183334,0.0,0.000000,0.0,0.000000,0.0,28.307196
ABCD1P2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF711,0.726268,2.033123,24.139853,6.579405,5.806381,37.119240,46.283546,33.331909,41.659931,10.467175,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,11.884506
ZNF847P,0.000000,0.000000,2.917047,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
ZNF886P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
ZNRF2,4.730500,0.000000,12.924584,11.742586,8.982515,17.712049,31.889036,15.987859,20.037987,29.351723,...,0.000000,2.286632,0.0,0.000000,0.0,0.338369,0.0,0.000000,0.0,0.000000


In [45]:
sample_meta_sum

,clusters,samples,SampleName
0,Elongating_spermatids,Crypto_1,clusters.Elongating_spermatids-samples.Crypto_1
1,Elongating_spermatids,Crypto_2,clusters.Elongating_spermatids-samples.Crypto_2
2,Elongating_spermatids,Crypto_3,clusters.Elongating_spermatids-samples.Crypto_3
3,Elongating_spermatids,Donor1_r1,clusters.Elongating_spermatids-samples.Donor1_r1
4,Elongating_spermatids,Donor1_r2,clusters.Elongating_spermatids-samples.Donor1_r2
...,...,...,...
190,Unknown_3,Normal_3,clusters.Unknown_3-samples.Normal_3
191,Unknown_3,OA,clusters.Unknown_3-samples.OA
192,Unknown_3,iNOA_1,clusters.Unknown_3-samples.iNOA_1
193,Unknown_3,iNOA_2,clusters.Unknown_3-samples.iNOA_2


In [54]:
sample_meta_sum['condition'] = sample_meta_sum.samples.map(map_condition)
sample_meta_sum

,clusters,samples,SampleName,condition
0,Elongating_spermatids,Crypto_1,clusters.Elongating_spermatids-samples.Crypto_1,Cryptozoospermia
1,Elongating_spermatids,Crypto_2,clusters.Elongating_spermatids-samples.Crypto_2,Cryptozoospermia
2,Elongating_spermatids,Crypto_3,clusters.Elongating_spermatids-samples.Crypto_3,Cryptozoospermia
3,Elongating_spermatids,Donor1_r1,clusters.Elongating_spermatids-samples.Donor1_r1,Normal
4,Elongating_spermatids,Donor1_r2,clusters.Elongating_spermatids-samples.Donor1_r2,Normal
...,...,...,...,...
190,Unknown_3,Normal_3,clusters.Unknown_3-samples.Normal_3,Normal
191,Unknown_3,OA,clusters.Unknown_3-samples.OA,obstructive_azoospermia
192,Unknown_3,iNOA_1,clusters.Unknown_3-samples.iNOA_1,Non-obstructive_azoospermia
193,Unknown_3,iNOA_2,clusters.Unknown_3-samples.iNOA_2,Non-obstructive_azoospermia


In [55]:
pseudobulk_matrix = pseudobulk_matrix_sum.T.to_csv(results_dir + "integrated_pseudobulk_matrix.csv")

In [56]:
pseudobulk_metadata = sample_meta_sum.to_csv(results_dir +'integrated_pseudobulk_metadata.csv')

# WGCNA